In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Post-Traumatic_Stress_Disorder/GSE77164'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Peripheral blood transcriptome profiles in Nepali child soldiers and civilians."
!Series_summary	"Analysis of transcript abundance estimates as a function of child soldier status, PTSD symptoms, and psychological resilience."
!Series_overall_design	"Gene expression profiling was conducted on dried blood spot (DBS) samples collected from community dwelling adolescents and young adults in Nepal.  Approximatley half of the sample were former child soldiers in the Nepal People's War and the other half were demographically similiar civilian non-combatants.  In addition to basic demographic characteristics (age, sex, ethnic minority status, social caste status, education level), participants were also assessed on syptoms of post-traumatic stress (PTS, assessed by a culturally adapted version of The Child PTSD Symptom Scale; Kohrt BA, et al. (2011) Validation of cross-cultural child mental health and psychosocial research instruments: adapting the Depres

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True

# Identify the keys for 'Post-Traumatic_Stress_Disorder', 'age', and 'gender'
trait_row = 6 if len(set(['pts: 0', 'pts: 1'])) > 1 else None
age_row = 2 if len(set(['age: 19', 'age: 22', 'age: 20', 'age: 18', 'age: 23', 'age: 21', 'age: 17', 'age: 16', 'age: 15', 'age: 24', 'age: 26'])) > 1 else None
gender_row = 1 if len(set(['female: 1', 'female: 0'])) > 1 else None

# Define the conversion functions
def convert_trait(value):
    try:
        val = int(value.split(':')[1])
        return val if val in [0, 1] else None
    except:
        return None

def convert_age(value):
    try:
        return float(value.split(':')[1])
    except:
        return None

def convert_gender(value):
    try:
        gender_val = int(value.split(':')[1])
        return 1 if gender_val == 0 else 0  # male: 1, female: 0
    except:
        return None

# Save metadata
save_cohort_info('GSE77164', './preprocessed/Post-Traumatic_Stress_Disorder/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Post-Traumatic_Stress_Disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Post-Traumatic_Stress_Disorder/trait_data/GSE77164.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM2045182': [0.0, 19.0, 0.0], 'GSM2045183': [0.0, 22.0, 1.0], 'GSM2045184': [0.0, 20.0, 1.0], 'GSM2045185': [0.0, 18.0, 1.0], 'GSM2045186': [0.0, 22.0, 1.0], 'GSM2045187': [0.0, 23.0, 0.0], 'GSM2045188': [1.0, 21.0, 1.0], 'GSM2045189': [0.0, 22.0, 0.0], 'GSM2045190': [0.0, 19.0, 0.0], 'GSM2045191': [0.0, 22.0, 1.0], 'GSM2045192': [0.0, 20.0, 1.0], 'GSM2045193': [0.0, 18.0, 1.0], 'GSM2045194': [0.0, 20.0, 1.0], 'GSM2045195': [0.0, 21.0, 0.0], 'GSM2045196': [0.0, 21.0, 1.0], 'GSM2045197': [0.0, 21.0, 1.0], 'GSM2045198': [0.0, 18.0, 1.0], 'GSM2045199': [0.0, 22.0, 1.0], 'GSM2045200': [0.0, 23.0, 1.0], 'GSM2045201': [1.0, 22.0, 0.0], 'GSM2045202': [0.0, 17.0, 0.0], 'GSM2045203': [0.0, 19.0, 1.0], 'GSM2045204': [1.0, 18.0, 0.0], 'GSM2045205': [0.0, 19.0, 0.0], 'GSM2045206': [0.0, 19.0, 0.0], 'GSM2045207': [0.0, 21.0, 1.0], 'GSM2045208': [0.0, 20.0, 1.0], 'GSM2045209': [0.0, 21.0, 1.0], 'GSM2045210': [1.0, 20.0, 0.0], 'GSM2045211': [0.0, 21.0, 1.0], 'GSM2045212': [0.0, 19.0, 1.0], 'GSM204

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1',
       'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AACSL', 'AADAC',
       'AADACL1', 'AADACL2', 'AADACL3', 'AADACL4'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# Since requires_gene_mapping is False as determined from STEP4, STEP6 is skipped


Gene annotation preview:
{'ID': ['7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1'], 'ORF': ['7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 7: Data Normalization and Merging

In [7]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Post-Traumatic_Stress_Disorder/gene_data/GSE77164.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Post-Traumatic_Stress_Disorder')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE77164', './preprocessed/Post-Traumatic_Stress_Disorder/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Post-Traumatic_Stress_Disorder/GSE77164.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Post-Traumatic_Stress_Disorder', the least common label is '1.0' with 29 occurrences. This represents 11.42% of the dataset.
The distribution of the feature 'Post-Traumatic_Stress_Disorder' in this dataset is fine.

Quartiles for 'Age':
  25%: 19.0
  50% (Median): 20.0
  75%: 21.75
Min: 15.0
Max: 26.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 113 occurrences. This represents 44.49% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

